# Vector-HaSH 类比推理模型 (e-kar数据集)

本notebook实现了基于多Grid的Vector-HaSH模型用于类比推理任务。

## 模型框架

- **多个Grid模块**：每个关系类型对应一个独立的Grid模块
- **共享HPC空间**：所有Grid共享同一个HPC（Place Cell）空间
- **双向连接**：
  - Grid ↔ HPC: 通过权重矩阵 W_pg 和 W_gp
  - HPC ↔ Sensory: 通过权重矩阵 W_sp 和 W_ps
- **Relation Classifier**：决定哪个Grid模块更新状态

## 生成过程

1. 随机生成gbook（Grid Cell表征）
2. 随机生成连接矩阵W_pg
3. 得到pbook = nonlin(gbook@Wpg)，全局共享
4. 通过伪逆学习得到W_gp


In [19]:
# === 导入库 ===
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import randn, randint
from tqdm import tqdm
from collections import defaultdict
import pandas as pd
import json
import os
import sys

# 导入自定义模块
sys.path.append(os.path.abspath('.'))
try:
    from src.assoc_utils_np import train_gcpc
    from src.assoc_utils_np_2D import gen_gbook_2d, path_integration_Wgg_2d, module_wise_NN_2d
    from src.seq_utils import nonlin, sensorymap
except ImportError:
    print("警告: 无法导入src模块，将使用本地实现")

# 绘图设置
plt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

print("✓ 导入完成")


✓ 导入完成


## 1. 超参数配置和模型结构初始化


In [20]:
# === 1. 基础结构超参数 ===
nruns = 1
Np = 342  # Place Cell数量
lambdas = [3, 4, 5, 7]  # Grid模块周期
Ng = np.sum(np.square(lambdas))  
Npos = np.prod(lambdas)  # Grid空间大小

print(f"Grid空间大小 (Npos): {Npos} x {Npos} = {Npos*Npos}")
print(f"Grid Cell数量 (Ng): {Ng}")
print(f"Place Cell数量 (Np): {Np}")

# === 2. 加载关系类型映射 ===
label_map_path = 'label_map.json'
with open(label_map_path, 'r', encoding='utf-8') as f:
    label_map = json.load(f)

# 关系类型列表（grid_index -> 关系名称）
RELATION_TYPES = [label_map[str(i)] for i in range(len(label_map))]
N_GRIDS = len(RELATION_TYPES)
RELATION_TO_IDX = {rel: idx for idx, rel in enumerate(RELATION_TYPES)}

print(f"\n=== 多Grid配置 ===")
print(f"Grid数量: {N_GRIDS}")
print(f"关系类型: {RELATION_TYPES[:5]}... (共{len(RELATION_TYPES)}种)")


Grid空间大小 (Npos): 420 x 420 = 176400
Grid Cell数量 (Ng): 99
Place Cell数量 (Np): 342

=== 多Grid配置 ===
Grid数量: 19
关系类型: ['Uncategorized', '主体-产物', '主体-动作', '亲属关系', '位置/空间']... (共19种)


## 2. 生成多个gbook和共享pbook


In [ ]:
# === 3. 生成多个gbook（每个关系类型一个） ===
gbooks = []
gbooks_flattened = []

for i in range(N_GRIDS):
    # 使用gen_gbook_2d生成2D Grid表征
    try:
        gbook = gen_gbook_2d(lambdas, Ng, Npos)
    except:
        # 如果导入失败，使用随机初始化
        gbook = randn(Ng, Npos, Npos)
        print(f"  警告: 使用随机初始化Grid '{RELATION_TYPES[i]}'")
    
    gbooks.append(gbook)
    gbooks_flattened.append(gbook.reshape(Ng, Npos*Npos))
    print(f"  Grid '{RELATION_TYPES[i]}' 生成完成: shape {gbook.shape}")

# === 4. 生成共享pbook（Place Cells）- 全局共享 ===
# 随机生成W_pg连接矩阵
Wpg = randn(nruns, Np, Ng)
c = 0.10  # 连接概率
prune = int((1-c)*Np*Ng)
mask = np.ones((Np, Ng))
mask[randint(low=0, high=Np, size=prune), randint(low=0, high=Ng, size=prune)] = 0
Wpg = np.multiply(mask, Wpg)
thresh = 2.0

# 使用第一个gbook作为参考生成pbook（全局共享）
# pbook = nonlin(gbook @ Wpg)
pbook = nonlin(np.einsum('ijk,klm->ijlm', Wpg, gbooks[0]), thresh=thresh)  #(nruns,Np,Npos,Npos)
pbook_flattened = pbook.reshape(nruns, Np, Npos*Npos)

print(f"\n✓ 共享HPC (pbook) 生成完成: shape {pbook.shape}")
print(f"  W_pg shape: {Wpg.shape}")

# === 5. 通过伪逆反向学习W_gp（为每个Grid维护对应的gbook和pbook） ===
# 虽然pbook是全局共享的，但每个Grid有自己的gbook，所以需要为每个Grid学习对应的W_gp
# W_gp: g = W_gp @ p，即 W_gp = g @ pinv(p)

print("\n通过伪逆反向学习W_gp...")
Wgp_list = []  # 为每个Grid存储W_gp

for grid_idx in range(N_GRIDS):
    # 将pbook和gbook展平为矩阵形式
    # pbook_flattened shape: (nruns, Np, Npos*Npos) -> (Np, Npos*Npos)
    # gbooks_flattened[grid_idx] shape: (Ng, Npos*Npos)
    
    # 直接使用展平后的矩阵
    P_matrix = pbook_flattened[0, :, :]  # (Np, Npos*Npos) - 所有位置的Place向量
    G_matrix = gbooks_flattened[grid_idx]  # (Ng, Npos*Npos) - 所有位置的Grid向量
        
    # 伪逆学习: W_gp = G @ pinv(P)
    try:
        P_pinv = np.linalg.pinv(P_matrix)
        Wgp_learned = G_matrix @ P_pinv  # (Ng, Np)
        Wgp_list.append(Wgp_learned)
        print(f"  Grid {grid_idx} ({RELATION_TYPES[grid_idx]}): W_gp学习完成, shape {Wgp_learned.shape}")
    except Exception as e:
        print(f"  Grid {grid_idx} 伪逆学习失败: {e}，使用零矩阵")
        Wgp_list.append(np.zeros((Ng, Np), dtype=float))

print(f"✓ W_gp学习完成，共 {len(Wgp_list)} 个Grid的W_gp矩阵")


  Grid 'Uncategorized' 生成完成: shape (99, 420, 420)
  Grid '主体-产物' 生成完成: shape (99, 420, 420)
  Grid '主体-动作' 生成完成: shape (99, 420, 420)
  Grid '亲属关系' 生成完成: shape (99, 420, 420)
  Grid '位置/空间' 生成完成: shape (99, 420, 420)
  Grid '包含/种属' 生成完成: shape (99, 420, 420)
  Grid '反义/对立' 生成完成: shape (99, 420, 420)
  Grid '因果/依赖' 生成完成: shape (99, 420, 420)
  Grid '属性/特征' 生成完成: shape (99, 420, 420)
  Grid '工具-功能' 生成完成: shape (99, 420, 420)
  Grid '师生传承' 生成完成: shape (99, 420, 420)
  Grid '并列/同类' 生成完成: shape (99, 420, 420)
  Grid '材料-成品' 生成完成: shape (99, 420, 420)
  Grid '等级/排序' 生成完成: shape (99, 420, 420)
  Grid '组成/整体' 生成完成: shape (99, 420, 420)
  Grid '职业-对象' 生成完成: shape (99, 420, 420)
  Grid '象征/比喻' 生成完成: shape (99, 420, 420)
  Grid '近义/同一' 生成完成: shape (99, 420, 420)
  Grid '顺序/过程' 生成完成: shape (99, 420, 420)

✓ 共享HPC (pbook) 生成完成: shape (1, 342, 420, 420)
  W_pg shape: (1, 342, 99)

通过伪逆反向学习W_gp...
  Grid 0 (Uncategorized): W_gp学习完成, shape (99, 342)
  Grid 1 (主体-产物): W_gp学习完成, shape (99, 342)
  Grid 2

## 3. 数据加载和预处理


In [22]:
# === 5. 加载e-kar数据 ===
data_path = 'obj_grid_index.csv'
df = pd.read_csv(data_path)

print(f"加载数据: {len(df)} 条词对")
print(f"数据预览:")
print(df.head(10))

# 统计每个grid的数据量
grid_counts = df['grid_index'].value_counts().sort_index()
print(f"\n各Grid数据量统计:")
for grid_idx, count in grid_counts.items():
    print(f"  Grid {grid_idx} ({RELATION_TYPES[grid_idx]}): {count} 条")

# 提取所有唯一的对象（用于生成Sensory向量）
all_objects = set(df['Obj_A'].tolist() + df['Obj_B'].tolist())
C = len(all_objects)  # 对象总数
print(f"\n唯一对象数量: {C}")


加载数据: 1675 条词对
数据预览:
  Obj_A Obj_B  grid_index
0    稻谷    大米           1
1    核桃    桃酥          12
2    棉花    棉籽           1
3    西瓜    瓜子           1
4    花生   花生酱           1
5  旗开得胜  马到成功          16
6  拨乱反正  沉冤昭雪          16
7  牛高马大  虎穴得子          16
8  水到渠成  瓜熟蒂落          16
9  缘木求鱼  鹰击长空          16

各Grid数据量统计:
  Grid 0 (Uncategorized): 41 条
  Grid 1 (主体-产物): 141 条
  Grid 2 (主体-动作): 294 条
  Grid 3 (亲属关系): 3 条
  Grid 4 (位置/空间): 89 条
  Grid 5 (包含/种属): 313 条
  Grid 6 (反义/对立): 115 条
  Grid 7 (因果/依赖): 102 条
  Grid 8 (属性/特征): 94 条
  Grid 9 (工具-功能): 85 条
  Grid 10 (师生传承): 3 条
  Grid 11 (并列/同类): 49 条
  Grid 12 (材料-成品): 53 条
  Grid 13 (等级/排序): 10 条
  Grid 14 (组成/整体): 47 条
  Grid 15 (职业-对象): 5 条
  Grid 16 (象征/比喻): 56 条
  Grid 17 (近义/同一): 127 条
  Grid 18 (顺序/过程): 48 条

唯一对象数量: 2641


## 4. Sensory向量生成和对象映射

使用中文embedding模型为每个对象生成语义向量。

**安装依赖**（如果尚未安装）：
```bash
pip install sentence-transformers
```

**使用的模型**：
- 主模型：`shibing624/text2vec-base-chinese` - 专门为中文语义相似度任务优化
- 备用模型：`paraphrase-multilingual-MiniLM-L12-v2` - 多语言模型
- 如果模型加载失败，将自动fallback到随机稀疏向量


In [ ]:
# === 6. 使用Embedding模型生成Sensory向量 ===
# 使用中文embedding模型为每个对象生成语义向量

try:
    from sentence_transformers import SentenceTransformer
    import torch
    
    # 检测可用设备
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"正在加载中文embedding模型... (设备: {device})")
    
    # 选择合适的中文embedding模型
    # 选项1: text2vec-base-chinese (专门为中文设计，适合语义相似度任务)
    # 选项2: bge-small-zh-v1.5 (更现代的中文embedding模型)
    # 选项3: paraphrase-multilingual-MiniLM-L12-v2 (多语言模型，支持中文)
    
    # 使用 text2vec-base-chinese，专门为中文语义相似度任务优化
    try:
        model_name = "shibing624/text2vec-base-chinese"
        print(f"  尝试加载模型: {model_name}")
        embedding_model = SentenceTransformer(model_name, device=device)
        print(f"  ✓ 成功加载模型: {model_name} (设备: {device})")
    except Exception as e1:
        print(f"  模型 {model_name} 加载失败: {e1}")
        try:
            # 备用选项：使用多语言模型
            model_name = "paraphrase-multilingual-MiniLM-L12-v2"
            print(f"  尝试加载备用模型: {model_name}")
            embedding_model = SentenceTransformer(model_name, device=device)
            print(f"  ✓ 成功加载备用模型: {model_name} (设备: {device})")
        except Exception as e2:
            print(f"  备用模型也加载失败: {e2}")
            print("  将使用随机向量作为fallback")
            embedding_model = None
    
    # 为每个对象生成Sensory向量
    object_to_id = {obj: idx for idx, obj in enumerate(sorted(all_objects))}
    id_to_object = {idx: obj for obj, idx in object_to_id.items()}
    events = []  # Sensory向量列表
    
    if embedding_model is not None:
        print(f"\n正在为 {C} 个对象生成embedding向量...")
        # 批量生成embedding（更高效）
        object_list = sorted(all_objects)
        embeddings = embedding_model.encode(
            object_list,
            batch_size=128,
            show_progress_bar=True,
            normalize_embeddings=True  # L2归一化
        )
        
        # 转换为numpy数组并存储
        for obj_id in range(C):
            event_vec = embeddings[obj_id].astype(np.float32)
            events.append(event_vec)
        
        Ns_demo = embeddings.shape[1]  # 实际embedding维度
        print(f"✓ 使用embedding模型生成 {C} 个对象的Sensory向量")
        print(f"  Embedding维度: {Ns_demo}")
        print(f"  模型: {model_name}")
    else:
        # Fallback: 使用随机向量
        raise ImportError("Embedding模型加载失败，使用随机向量")
        
except ImportError as e:
    print(f"警告: 无法导入sentence_transformers库: {e}")
    print("请安装: pip install sentence-transformers")
    print("将使用随机稀疏向量作为fallback...")
    
    # Fallback: 使用随机稀疏向量
    rng = np.random.default_rng(42)
    Ns_demo = 512  # Sensory维度
    k_sparse = 12  # 稀疏度
    
    def make_sparse_vec(N, k_sparse, rng):
        """生成随机稀疏向量"""
        v = np.zeros(N, dtype=float)
        idx = rng.choice(N, size=k_sparse, replace=False)
        v[idx] = rng.choice([-1.0, 1.0], size=k_sparse)
        v /= (np.linalg.norm(v) + 1e-12)
        return v
    
    object_to_id = {obj: idx for idx, obj in enumerate(sorted(all_objects))}
    id_to_object = {idx: obj for obj, idx in object_to_id.items()}
    events = []
    
    for obj_id in range(C):
        event_vec = make_sparse_vec(Ns_demo, k_sparse, rng)
        events.append(event_vec)
    
    print(f"✓ 生成 {C} 个对象的随机Sensory向量 (维度={Ns_demo}, 稀疏度={k_sparse})")

except Exception as e:
    print(f"错误: {e}")
    print("将使用随机稀疏向量作为fallback...")
    
    # Fallback: 使用随机稀疏向量
    rng = np.random.default_rng(42)
    Ns_demo = 512
    k_sparse = 12
    
    def make_sparse_vec(N, k_sparse, rng):
        v = np.zeros(N, dtype=float)
        idx = rng.choice(N, size=k_sparse, replace=False)
        v[idx] = rng.choice([-1.0, 1.0], size=k_sparse)
        v /= (np.linalg.norm(v) + 1e-12)
        return v
    
    object_to_id = {obj: idx for idx, obj in enumerate(sorted(all_objects))}
    id_to_object = {idx: obj for obj, idx in object_to_id.items()}
    events = []
    
    for obj_id in range(C):
        event_vec = make_sparse_vec(Ns_demo, k_sparse, rng)
        events.append(event_vec)
    
    print(f"✓ 生成 {C} 个对象的随机Sensory向量 (维度={Ns_demo}, 稀疏度={k_sparse})")

print(f"\n对象映射示例: {list(object_to_id.items())[:5]}")
print(f"Sensory向量维度: {len(events[0]) if events else 'N/A'}")


正在加载中文embedding模型... (设备: cuda)
  尝试加载模型: shibing624/text2vec-base-chinese
  ✓ 成功加载模型: shibing624/text2vec-base-chinese (设备: cuda)

正在为 2641 个对象生成embedding向量...


Batches: 100%|██████████| 21/21 [00:00<00:00, 113.11it/s]

✓ 使用embedding模型生成 2641 个对象的Sensory向量
  Embedding维度: 768
  模型: shibing624/text2vec-base-chinese

对象映射示例: [('B超', 0), ('CT机', 1), ('GPS', 2), ('QQ', 3), ('X光片', 4)]
Sensory向量维度: 768


In [42]:

print(np.array(events).shape)

(2641, 768)


## 5. 辅助函数定义


In [24]:
# === 7. 辅助函数 ===

def flat_idx(x, y, Npos=Npos):
    """将2D坐标转换为1D索引"""
    return int(x * Npos + y)

def get_pvec(x, y, pbook_flattened=pbook_flattened):
    """获取指定坐标的Place Cell向量"""
    return pbook_flattened[0, :, flat_idx(x, y)]

def cosine_sim(v1, v2):
    """计算余弦相似度"""
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2) + 1e-9)

def rls_step(W, theta, a, y):
    """
    递归最小二乘(RLS)更新步骤
    """
    a = np.asarray(a).reshape(-1, 1)
    y = np.asarray(y).reshape(-1, 1)
    
    pred_before = W @ a
    err_before = y - pred_before
    
    denom = 1.0 + (a.T @ theta @ a).item()
    bk = (theta @ a) / denom
    
    theta_new = theta - (theta @ a @ bk.T)
    W_new = W + (err_before @ bk.T)
    
    pred_after = W_new @ a
    err_after = y - pred_after
    
    return W_new, theta_new, float(np.linalg.norm(bk)), float(np.linalg.norm(err_before)), float(np.linalg.norm(err_after))

print("✓ 辅助函数定义完成")


✓ 辅助函数定义完成


## 6. 权重矩阵初始化（W_gp, W_ps, W_sp）


In [ ]:
# === 8. 初始化权重矩阵 ===
epsilon = 0.05
# W_ps:  Sensory -> Place 
Wps = np.zeros((Np, Ns_demo), dtype=float)
theta_ps = (1.0 / (epsilon**2)) * np.eye(Ns_demo)

# W_sp:  Place -> Sensory
Wsp = np.zeros((Ns_demo, Np), dtype=float)
theta_sp = (1.0 / (epsilon**2)) * np.eye(Np)

print(f"✓ 权重矩阵初始化完成")
print(f"  W_gp: {len(Wgp_list)} 个矩阵，每个 shape {Wgp_list[0].shape}")
print(f"  W_ps: shape {Wps.shape}")
print(f"  W_sp: shape {Wsp.shape}")


  使用已初始化的Wgp_list，共 19 个Grid
✓ 权重矩阵初始化完成
  W_gp: 19 个矩阵，每个 shape (99, 342)
  W_ps: shape (342, 768)
  W_sp: shape (768, 342)


## 7. 对象到HPC位置的映射（全局唯一位置分配）


In [26]:
# === 10. 全局HPC位置分配 ===
# 每个对象在HPC中只有一个全局唯一位置（与Grid无关）

object_hpc_locs = {}  # object_id -> (x, y)
hpc_next_x = 0
hpc_next_y = 0
hpc_spacing = 3  # 对象之间的间隔

def allocate_global_hpc_position(obj_id):
    """为对象分配全局唯一的HPC位置"""
    global hpc_next_x, hpc_next_y
    
    if obj_id in object_hpc_locs:
        return object_hpc_locs[obj_id]
    
    x, y = hpc_next_x, hpc_next_y
    object_hpc_locs[obj_id] = (x, y)
    
    hpc_next_x += hpc_spacing
    if hpc_next_x >= Npos:
        hpc_next_x = 0
        hpc_next_y += hpc_spacing
    
    return (x, y)

def check_hpc_space_enough(num_objects):
    """
    检测HPC空间是否足够容纳指定数量的对象
    :param num_objects: 需要分配位置的对象总数
    :return: True表示空间足够，False表示空间不足
    """
    # 计算每行能容纳的对象数量
    objects_per_row = (Npos-1) // hpc_spacing + 1
    
    objects_per_column = objects_per_row

    total_available_num = objects_per_row * objects_per_column

    if num_objects > total_available_num:
        return False
    else:
        return True

# 1. 先检测空间是否足够
if not check_hpc_space_enough(C):
    print(f"【错误】HPC空间不足，无法为{C}个对象分配位置！")
else:
    # 2. 空间足够时才分配位置
    for obj_id in range(C):
        allocate_global_hpc_position(obj_id)
    print(f"【成功】已为{C}个对象分配HPC位置，位置信息：{object_hpc_locs}")


【成功】已为2641个对象分配HPC位置，位置信息：{0: (0, 0), 1: (3, 0), 2: (6, 0), 3: (9, 0), 4: (12, 0), 5: (15, 0), 6: (18, 0), 7: (21, 0), 8: (24, 0), 9: (27, 0), 10: (30, 0), 11: (33, 0), 12: (36, 0), 13: (39, 0), 14: (42, 0), 15: (45, 0), 16: (48, 0), 17: (51, 0), 18: (54, 0), 19: (57, 0), 20: (60, 0), 21: (63, 0), 22: (66, 0), 23: (69, 0), 24: (72, 0), 25: (75, 0), 26: (78, 0), 27: (81, 0), 28: (84, 0), 29: (87, 0), 30: (90, 0), 31: (93, 0), 32: (96, 0), 33: (99, 0), 34: (102, 0), 35: (105, 0), 36: (108, 0), 37: (111, 0), 38: (114, 0), 39: (117, 0), 40: (120, 0), 41: (123, 0), 42: (126, 0), 43: (129, 0), 44: (132, 0), 45: (135, 0), 46: (138, 0), 47: (141, 0), 48: (144, 0), 49: (147, 0), 50: (150, 0), 51: (153, 0), 52: (156, 0), 53: (159, 0), 54: (162, 0), 55: (165, 0), 56: (168, 0), 57: (171, 0), 58: (174, 0), 59: (177, 0), 60: (180, 0), 61: (183, 0), 62: (186, 0), 63: (189, 0), 64: (192, 0), 65: (195, 0), 66: (198, 0), 67: (201, 0), 68: (204, 0), 69: (207, 0), 70: (210, 0), 71: (213, 0), 72: (216, 0),

## 8.学习Wps和Wsp

In [27]:
# === 11. 训练循环：提取C个对象的Place Cell向量到path_pbook ===
Npatts = C  # 对象数量
path_pbook = np.zeros((nruns, Np, Npatts))

# 循环提取每个对象在其HPC位置的Place Cell向量
for obj_id in range(C):
    x, y = object_hpc_locs[obj_id]
    path_pbook[0, :, obj_id] = pbook_flattened[0, :, flat_idx(x, y)]

print(f"✓ 提取完成: path_pbook shape {path_pbook.shape}，包含{C}个对象的Place向量")

# === 12. 伪逆学习 W_ps 和 W_sp ===
# 利用path_pbook (Place向量) 和 events (Sensory向量) 学习映射

# 将events列表转换为矩阵形式 (Ns_demo, C)
events_matrix = np.array(events).T  # shape: (Ns_demo, C)

# 提取path_pbook矩阵 (Np, C)
path_pbook_matrix = path_pbook[0, :, :]  # shape: (Np, C)

print(f"\n开始伪逆学习W_ps和W_sp...")
print(f"  events_matrix shape: {events_matrix.shape}")
print(f"  path_pbook_matrix shape: {path_pbook_matrix.shape}")

# 学习 W_ps: Sensory -> Place
# p = W_ps @ s，即 W_ps = path_pbook @ pinv(events)
try:
    events_pinv = np.linalg.pinv(events_matrix)
    Wps = path_pbook_matrix @ events_pinv  # shape: (Np, Ns_demo)
    print(f"  ✓ W_ps学习成功, shape {Wps.shape}")
except Exception as e:
    print(f"  ✗ W_ps伪逆学习失败: {e}")
    Wps = np.zeros((Np, Ns_demo), dtype=float)

# 学习 W_sp: Place -> Sensory
# s = W_sp @ p，即 W_sp = events @ pinv(path_pbook)
try:
    path_pbook_pinv = np.linalg.pinv(path_pbook_matrix)
    Wsp = events_matrix @ path_pbook_pinv  # shape: (Ns_demo, Np)
    print(f"  ✓ W_sp学习成功, shape {Wsp.shape}")
except Exception as e:
    print(f"  ✗ W_sp伪逆学习失败: {e}")
    Wsp = np.zeros((Ns_demo, Np), dtype=float)

print(f"\n✓ W_ps和W_sp伪逆学习完成")


✓ 提取完成: path_pbook shape (1, 342, 2641)，包含2641个对象的Place向量

开始伪逆学习W_ps和W_sp...
  events_matrix shape: (768, 2641)
  path_pbook_matrix shape: (342, 2641)
  ✓ W_ps学习成功, shape (342, 768)
  ✓ W_sp学习成功, shape (768, 342)

✓ W_ps和W_sp伪逆学习完成


In [28]:
# === 13. 初始化对象在gbook中的位置映射（使用HPC位置） ===
print("\n初始化对象在各Grid的gbook中的位置映射...")
print("  初始位置映射与HPC位置相同（所有Grid一致）")

# gbook_obj_positions[grid_idx][obj_id] = (x, y) 记录obj在该Grid的gbook(i)中的位置
gbook_obj_positions = {}  
# gbook_pos_obj[grid_idx][flat_idx(x,y)] = obj_id 记录该位置上是哪个对象
gbook_pos_obj = {}  

# 为每个Grid初始化位置映射（使用HPC位置）
for grid_idx in range(N_GRIDS):
    gbook_obj_positions[grid_idx] = {}
    gbook_pos_obj[grid_idx] = {}
    
    # 为每个对象使用HPC位置来初始化
    for obj_id in range(C):
        # 从HPC位置获取该对象的坐标
        x, y = object_hpc_locs[obj_id]
        flat_i = flat_idx(x, y)
        
        # 在所有Grid中都使用相同的HPC位置
        gbook_obj_positions[grid_idx][obj_id] = (x, y)
        gbook_pos_obj[grid_idx][flat_i] = obj_id

print(f"✓ 位置映射初始化完成")



初始化对象在各Grid的gbook中的位置映射...
  初始位置映射与HPC位置相同（所有Grid一致）
✓ 位置映射初始化完成


In [65]:
# === 14. 使用RLS在线学习更新W_gp[i]和Wpg[i]（包含gbook位置交换） ===
print("\n开始训练：调整gbook位置并使用RLS更新W_gp和Wpg...")

# 为每个Grid初始化RLS的协方差矩阵
theta_gp_list = [(1.0 / (epsilon**2)) * np.eye(Np) for _ in range(N_GRIDS)]
# 为每个Grid初始化W_pg的协方差矩阵 (Ng维度)
theta_pg_list = [(1.0 / (epsilon**2)) * np.eye(Ng) for _ in range(N_GRIDS)]

# 将W_pg扩展为每个Grid一份（初始化为原始的Wpg）
Wpg_list = []
for grid_idx in range(N_GRIDS):
    Wpg_grid = Wpg[0, :, :].copy()
    Wpg_list.append(Wpg_grid)

print(f"初始化 {N_GRIDS} 个Grid的Wpg矩阵")

# 为每个Grid维护位置计数器
pair_position_counter = {i: 0 for i in range(N_GRIDS)}

def allocate_pair_position(grid_idx, pair_position_counter, Npos):
    """
    为数据对分配目标位置
    :param grid_idx: Grid索引
    :param pair_position_counter: 位置计数器字典
    :param Npos: 网格空间大小
    :return: 返回(x_a, y_a, x_b, y_b)坐标
    """
    pair_pos = pair_position_counter[grid_idx]
    
    # 每行可以放Npos//2对（每对占2个单位）
    pairs_per_row = Npos // 2
    row_idx = pair_pos // pairs_per_row
    col_in_row = pair_pos % pairs_per_row
    
    x = col_in_row * 2
    y = row_idx
    
    return x, y, x + 1, y

def swap_gbook_vectors(grid_idx, obj_a_id, obj_b_id, target_x_a, target_y_a, target_x_b, target_y_b):
    """
    交换gbook中的向量并更新位置映射
    :return: 返回新的位置信息
    """
    # 获取当前位置
    curr_pos_a = gbook_obj_positions[grid_idx].get(obj_a_id)
    curr_pos_b = gbook_obj_positions[grid_idx].get(obj_b_id)

    curr_flat_a = flat_idx(*curr_pos_a)
    curr_flat_b = flat_idx(*curr_pos_b)
    target_flat_a = flat_idx(target_x_a, target_y_a)
    target_flat_b = flat_idx(target_x_b, target_y_b)
    
    # 获取目标位置上原有的对象
    obj_at_target_a = gbook_pos_obj[grid_idx].get(target_flat_a)
    obj_at_target_b = gbook_pos_obj[grid_idx].get(target_flat_b)
    
    # ===== 交换 obj_a 的位置 =====
    # 交换gbook中两处的向量
    temp_vec = gbooks_flattened[grid_idx][:, curr_flat_a].copy()
    gbooks_flattened[grid_idx][:, curr_flat_a] = gbooks_flattened[grid_idx][:, target_flat_a]
    gbooks_flattened[grid_idx][:, target_flat_a] = temp_vec
    
    # 更新位置映射
    gbook_obj_positions[grid_idx][obj_a_id] = (target_x_a, target_y_a)
    gbook_pos_obj[grid_idx][target_flat_a] = obj_a_id
    gbook_obj_positions[grid_idx][obj_at_target_a] = curr_pos_a
    gbook_pos_obj[grid_idx][curr_flat_a] = obj_at_target_a
    
    # ===== 交换 obj_b 的位置 =====
    # 交换gbook中两处的向量
    temp_vec = gbooks_flattened[grid_idx][:, curr_flat_b].copy()
    gbooks_flattened[grid_idx][:, curr_flat_b] = gbooks_flattened[grid_idx][:, target_flat_b]
    gbooks_flattened[grid_idx][:, target_flat_b] = temp_vec
    
    # 更新位置映射
    gbook_obj_positions[grid_idx][obj_b_id] = (target_x_b, target_y_b)
    gbook_pos_obj[grid_idx][target_flat_b] = obj_b_id
    gbook_obj_positions[grid_idx][obj_at_target_b] = curr_pos_b
    gbook_pos_obj[grid_idx][curr_flat_b] = obj_at_target_b

    # 返回新位置
    return target_x_a, target_y_a, target_x_b, target_y_b

# 迭代训练数据
train_count = 0
for idx, row in df.iterrows():
    obj_a = row['Obj_A']
    obj_b = row['Obj_B']
    grid_idx = int(row['grid_index'])
    
    # 获取对象ID
    obj_a_id = object_to_id.get(obj_a)
    obj_b_id = object_to_id.get(obj_b)
    
    if obj_a_id is None or obj_b_id is None:
        continue
    
    # 分配目标位置
    target_x_a, target_y_a, target_x_b, target_y_b = allocate_pair_position(grid_idx, pair_position_counter, Npos)
    
    # 检查位置是否超出边界
    if target_y_a >= Npos or target_y_b >= Npos:
        print(f"  警告: Grid {grid_idx} ({RELATION_TYPES[grid_idx]}) 位置空间不足")
        break
    
    # 交换gbook中的向量，更新位置映射
    new_x_a, new_y_a, new_x_b, new_y_b = swap_gbook_vectors(
        grid_idx, obj_a_id, obj_b_id, target_x_a, target_y_a, target_x_b, target_y_b
    )
    
    #获取id
    idx_a = flat_idx(new_x_a, new_y_a)
    idx_b = flat_idx(new_x_b, new_y_b)
    
    # 从新位置提取Grid向量
    g_a = gbooks_flattened[grid_idx][:, idx_a]
    g_b = gbooks_flattened[grid_idx][:, idx_b]
    
    # 提取place向量
    x_a, y_a = object_hpc_locs[obj_a_id]
    x_b, y_b = object_hpc_locs[obj_b_id]
    
    idx_p_a = flat_idx(x_a, y_a)
    idx_p_b = flat_idx(x_b, y_b)
    p_a = pbook_flattened[0, :, idx_p_a]
    p_b = pbook_flattened[0, :, idx_p_b]
    
    # ===== 使用RLS更新 W_gp[grid_idx] =====
    # g = W_gp @ p
    Wgp_list[grid_idx], theta_gp_list[grid_idx], _, _, _ = rls_step(
        Wgp_list[grid_idx], theta_gp_list[grid_idx], p_a, g_a
    )
    Wgp_list[grid_idx], theta_gp_list[grid_idx], _, _, _ = rls_step(
        Wgp_list[grid_idx], theta_gp_list[grid_idx], p_b, g_b
    )
    
    # ===== 使用RLS更新 W_pg[grid_idx] =====
    # p = W_pg @ g
    Wpg_list[grid_idx], theta_pg_list[grid_idx], _, _, _ = rls_step(
        Wpg_list[grid_idx], theta_pg_list[grid_idx], g_a, p_a
    )
    Wpg_list[grid_idx], theta_pg_list[grid_idx], _, _, _ = rls_step(
        Wpg_list[grid_idx], theta_pg_list[grid_idx], g_b, p_b
    )
    
    train_count += 1
    pair_position_counter[grid_idx] += 1
    
    if (train_count + 1) % 500 == 0:
        print(f"  已处理 {train_count} 条训练数据")
        # 显示某个Grid的位置分布示例
        sample_grid = 0
        sample_positions = list(gbook_obj_positions[sample_grid].values())[:5]
        print(f"    (Grid {sample_grid}样本位置: {sample_positions})")

print(f"\n✓ 训练完成，共处理 {train_count} 条数据")
print(f"  W_gp列表已更新，共 {len(Wgp_list)} 个Grid模块")
print(f"  W_pg列表已更新，共 {len(Wpg_list)} 个Grid模块")
print(f"\n各Grid的数据处理统计:")
for grid_idx in range(N_GRIDS):
    pair_count = pair_position_counter[grid_idx]
    if pair_count > 0:
        pairs_per_row = Npos // 2
        last_row = (pair_count - 1) // pairs_per_row
        print(f"  Grid {grid_idx} ({RELATION_TYPES[grid_idx]}): {pair_count}对数据，占用行0-{last_row}")



开始训练：调整gbook位置并使用RLS更新W_gp和Wpg...
初始化 19 个Grid的Wpg矩阵
  已处理 499 条训练数据
    (Grid 0样本位置: [(342, 6), (84, 21), (336, 3), (270, 39), (258, 42)])
  已处理 999 条训练数据
    (Grid 0样本位置: [(342, 6), (84, 21), (336, 3), (270, 39), (258, 42)])
  已处理 1499 条训练数据
    (Grid 0样本位置: [(342, 6), (84, 21), (336, 3), (270, 39), (258, 42)])

✓ 训练完成，共处理 1675 条数据
  W_gp列表已更新，共 19 个Grid模块
  W_pg列表已更新，共 19 个Grid模块

各Grid的数据处理统计:
  Grid 0 (Uncategorized): 41对数据，占用行0-0
  Grid 1 (主体-产物): 141对数据，占用行0-0
  Grid 2 (主体-动作): 294对数据，占用行0-1
  Grid 3 (亲属关系): 3对数据，占用行0-0
  Grid 4 (位置/空间): 89对数据，占用行0-0
  Grid 5 (包含/种属): 313对数据，占用行0-1
  Grid 6 (反义/对立): 115对数据，占用行0-0
  Grid 7 (因果/依赖): 102对数据，占用行0-0
  Grid 8 (属性/特征): 94对数据，占用行0-0
  Grid 9 (工具-功能): 85对数据，占用行0-0
  Grid 10 (师生传承): 3对数据，占用行0-0
  Grid 11 (并列/同类): 49对数据，占用行0-0
  Grid 12 (材料-成品): 53对数据，占用行0-0
  Grid 13 (等级/排序): 10对数据，占用行0-0
  Grid 14 (组成/整体): 47对数据，占用行0-0
  Grid 15 (职业-对象): 5对数据，占用行0-0
  Grid 16 (象征/比喻): 56对数据，占用行0-0
  Grid 17 (近义/同一): 127对数据，占用行0-0
  Grid 18 (顺序/过程): 48对数

In [17]:
def nearest_neighbor(gin, gbook):
    
    # 计算相似度: (n_positions,)
    similarities = gin @ gbook
    
    # 找到所有最大值的索引（处理平局）
    max_val = np.max(similarities)
    max_indices = np.argwhere(similarities == max_val).flatten()  # 所有最大值位置
    
    # 随机选一个
    idx = np.random.choice(max_indices)
    
    return gbook[:, idx]

In [63]:
#离线一次性更新===
print("\n开始训练：调整gbook位置并使用RLS更新W_gp和Wpg...")

# 将W_pg扩展为每个Grid一份（初始化为原始的Wpg）
Wpg_list = []
for grid_idx in range(N_GRIDS):
    Wpg_grid = Wpg[0, :, :].copy()
    Wpg_list.append(Wpg_grid)

print(f"初始化 {N_GRIDS} 个Grid的Wpg矩阵")

# 为每个Grid维护位置计数器
pair_position_counter = {i: 0 for i in range(N_GRIDS)}

def allocate_pair_position(grid_idx, pair_position_counter, Npos):
    """
    为数据对分配目标位置
    :param grid_idx: Grid索引
    :param pair_position_counter: 位置计数器字典
    :param Npos: 网格空间大小
    :return: 返回(x_a, y_a, x_b, y_b)坐标
    """
    pair_pos = pair_position_counter[grid_idx]
    
    # 每行可以放Npos//2对（每对占2个单位）
    pairs_per_row = Npos // 2
    row_idx = pair_pos // pairs_per_row
    col_in_row = pair_pos % pairs_per_row
    
    x = col_in_row * 2
    y = row_idx
    
    return x, y, x + 1, y

def swap_gbook_vectors(grid_idx, obj_a_id, obj_b_id, target_x_a, target_y_a, target_x_b, target_y_b):
    """
    交换gbook中的向量并更新位置映射
    :return: 返回新的位置信息
    """
    # 获取当前位置
    curr_pos_a = gbook_obj_positions[grid_idx].get(obj_a_id)
    curr_pos_b = gbook_obj_positions[grid_idx].get(obj_b_id)

    curr_flat_a = flat_idx(*curr_pos_a)
    curr_flat_b = flat_idx(*curr_pos_b)
    target_flat_a = flat_idx(target_x_a, target_y_a)
    target_flat_b = flat_idx(target_x_b, target_y_b)
    
    # 获取目标位置上原有的对象
    obj_at_target_a = gbook_pos_obj[grid_idx].get(target_flat_a)
    obj_at_target_b = gbook_pos_obj[grid_idx].get(target_flat_b)
    
    # ===== 交换 obj_a 的位置 =====
    # 交换gbook中两处的向量
    temp_vec = gbooks_flattened[grid_idx][:, curr_flat_a].copy()
    gbooks_flattened[grid_idx][:, curr_flat_a] = gbooks_flattened[grid_idx][:, target_flat_a]
    gbooks_flattened[grid_idx][:, target_flat_a] = temp_vec
    
    # 更新位置映射
    gbook_obj_positions[grid_idx][obj_a_id] = (target_x_a, target_y_a)
    gbook_pos_obj[grid_idx][target_flat_a] = obj_a_id
    gbook_obj_positions[grid_idx][obj_at_target_a] = curr_pos_a
    gbook_pos_obj[grid_idx][curr_flat_a] = obj_at_target_a
    
    # ===== 交换 obj_b 的位置 =====
    # 交换gbook中两处的向量
    temp_vec = gbooks_flattened[grid_idx][:, curr_flat_b].copy()
    gbooks_flattened[grid_idx][:, curr_flat_b] = gbooks_flattened[grid_idx][:, target_flat_b]
    gbooks_flattened[grid_idx][:, target_flat_b] = temp_vec
    
    # 更新位置映射
    gbook_obj_positions[grid_idx][obj_b_id] = (target_x_b, target_y_b)
    gbook_pos_obj[grid_idx][target_flat_b] = obj_b_id
    gbook_obj_positions[grid_idx][obj_at_target_b] = curr_pos_b
    gbook_pos_obj[grid_idx][curr_flat_b] = obj_at_target_b

    # 返回新位置
    return target_x_a, target_y_a, target_x_b, target_y_b


for idx, row in df.iterrows():
    obj_a = row['Obj_A']
    obj_b = row['Obj_B']
    grid_idx = int(row['grid_index'])
    
    # 获取对象ID
    obj_a_id = object_to_id.get(obj_a)
    obj_b_id = object_to_id.get(obj_b)
    
    if obj_a_id is None or obj_b_id is None:
        continue
    
    # 分配目标位置
    target_x_a, target_y_a, target_x_b, target_y_b = allocate_pair_position(grid_idx, pair_position_counter, Npos)
    
    # 检查位置是否超出边界
    if target_y_a >= Npos or target_y_b >= Npos:
        print(f"  警告: Grid {grid_idx} ({RELATION_TYPES[grid_idx]}) 位置空间不足")
        break
    
    # 交换gbook中的向量，更新位置映射
    new_x_a, new_y_a, new_x_b, new_y_b = swap_gbook_vectors(
        grid_idx, obj_a_id, obj_b_id, target_x_a, target_y_a, target_x_b, target_y_b
    )
    
    #获取id
    idx_a = flat_idx(new_x_a, new_y_a)
    idx_b = flat_idx(new_x_b, new_y_b)
    
    # 从新位置提取Grid向量
    g_a = gbooks_flattened[grid_idx][:, idx_a]
    g_b = gbooks_flattened[grid_idx][:, idx_b]
    
    # 提取place向量
    x_a, y_a = object_hpc_locs[obj_a_id]
    x_b, y_b = object_hpc_locs[obj_b_id]
    
    idx_p_a = flat_idx(x_a, y_a)
    idx_p_b = flat_idx(x_b, y_b)
    p_a = pbook_flattened[0, :, idx_p_a]
    p_b = pbook_flattened[0, :, idx_p_b]
    
    train_count += 1
    pair_position_counter[grid_idx] += 1
    
    if (train_count + 1) % 500 == 0:
        print(f"  已处理 {train_count} 条训练数据")
        # 显示某个Grid的位置分布示例
        sample_grid = 0
        sample_positions = list(gbook_obj_positions[sample_grid].values())[:5]
        print(f"    (Grid {sample_grid}样本位置: {sample_positions})")

for grid_idx in range(N_GRIDS):
    pbook_flattened_pinv = np.linalg.pinv(pbook_flattened)
    Wgp_list[grid_idx] = gbooks_flattened[grid_idx] @ pbook_flattened_pinv
for grid_idx in range(N_GRIDS):
    gbooks_flattened_pinv = np.linalg.pinv(gbooks_flattened[grid_idx])
    Wpg_list[grid_idx] = pbook_flattened @ gbooks_flattened_pinv



开始训练：调整gbook位置并使用RLS更新W_gp和Wpg...
初始化 19 个Grid的Wpg矩阵
  已处理 1999 条训练数据
    (Grid 0样本位置: [(342, 6), (84, 21), (336, 3), (270, 39), (258, 42)])
  已处理 2499 条训练数据
    (Grid 0样本位置: [(342, 6), (84, 21), (336, 3), (270, 39), (258, 42)])
  已处理 2999 条训练数据
    (Grid 0样本位置: [(342, 6), (84, 21), (336, 3), (270, 39), (258, 42)])


## 测试VH权重连接


In [18]:
import pandas as pd
# === 测试VH权重连接：前10个例子 ===
print('='*70)
print('测试VH权重连接：检测obj_grid_index前9个例子')
print('='*70)

df = pd.read_csv("obj_grid_index.csv")
# 读取前10个例子
test_samples = df.head(9)
print(f'测试样本数: {len(test_samples)}')
print(test_samples[['Obj_A', 'Obj_B', 'grid_index']])

# 迭代参数
Niter = 5
module_sizes = np.square(lambdas)  # [9, 16, 25, 49]
module_gbooks = [np.eye(i) for i in module_sizes]
m = len(lambdas)

print(f'迭代参数:')
print(f'  Niter = {Niter}')
print(f'  module_sizes = {module_sizes}')
print(f'  m = {m}')

# 存储结果
results = []

print('' + '='*70)
print('开始逐个测试')
print('='*70)

for idx, row in test_samples.iterrows():
    obj_a = row['Obj_A']
    obj_b = row['Obj_B']
    grid_idx = int(row['grid_index'])
    
    print(f'【样本 {idx+1}】')
    print(f'  Obj_A: {obj_a}')
    print(f'  Obj_B: {obj_b}')
    print(f'  Grid: {grid_idx} ({RELATION_TYPES[grid_idx]})')
    
    # 1. 获取obj_a的embedding (s_input)
    try:
        s_input = embedding_model.encode(obj_a, normalize_embeddings=True).astype(np.float32)
        print(f'  ✓ s_input shape: {s_input.shape}, norm: {np.linalg.norm(s_input):.4f}')
    except Exception as e:
        print(f'  ✗ 获取embedding失败: {e}')
        continue
    
    # 2. 通过Wps得到p_input
    p_input = Wps @ s_input  # shape: (Np,)
    print(f'  ✓ p_input shape: {p_input.shape}, norm: {np.linalg.norm(p_input):.4f}')
    
    # 3. 提取全局HPC的P_true (obj_a的真实place向量)
    obj_a_id = object_to_id.get(obj_a)
    if obj_a_id is None:
        print(f'  ✗ 对象 {obj_a} 不在词汇表中')
        continue
    
    x_a, y_a = object_hpc_locs[obj_a_id]
    P_true = pbook_flattened[0, :, flat_idx(x_a, y_a)]  # shape: (Np,)
    print(f'  ✓ P_true shape: {P_true.shape}, norm: {np.linalg.norm(P_true):.4f}')
    
    # 4. 计算p_input和P_true的余弦相似度
    cos_sim_p_init = cosine_sim(p_input, P_true)
    print(f'  ✓ 初始p余弦相似度: {cos_sim_p_init:.4f}')
    
    # 5. 获取obj_a在gbook中的真实g向量
    if obj_a_id in gbook_obj_positions[grid_idx]:
        gbook_pos = gbook_obj_positions[grid_idx][obj_a_id]
        g_true = gbooks_flattened[grid_idx][:, flat_idx(*gbook_pos)]
        print(f'  ✓ g_true shape: {g_true.shape}, norm: {np.linalg.norm(g_true):.4f}')
    else:
        print(f'  ✗ 对象 {obj_a} 在Grid {grid_idx} 中没有位置')
        continue
    
    # 6. 获取obj_b的真实sensory向量
    obj_b_id = object_to_id.get(obj_b)
    if obj_b_id is None:
        print(f'  ✗ 对象 {obj_b} 不在词汇表中')
        continue
    s_true = events[obj_b_id]
    print(f'  ✓ s_true (obj_b) shape: {s_true.shape}, norm: {np.linalg.norm(s_true):.4f}')
    
    # 7. 迭代优化
    print(f'  开始迭代优化 (Niter={Niter}):')
    
    # 使用当前Grid的Wgp和Wpg
    Wgp = Wgp_list[grid_idx]  # shape: (Ng, Np)
    Wpg = Wpg_list[grid_idx]  # shape: (Np, Ng)
    
    
    # 获取当前grid_idx对应的gbook
    gbook_current = gbooks_flattened[grid_idx]  # shape: (Ng, n_positions)
    p = p_input.copy()
    
    for i in range(Niter):
        # g_in = Wgp @ p
        g_in = Wgp @ p
        
        # 使用nearest_neighbor函数，针对当前grid的gbook进行最近邻搜索
        g = nearest_neighbor(g_in, gbook_current)  # 返回 shape: (1, Ng)
        g = g.flatten()  # 转为 (Ng,)
        
        # p = nonlin(Wpg @ g, thresh)
        p = nonlin(Wpg @ g, thresh)
        
        # 计算误差
        g_error = np.linalg.norm(g - g_true) / (np.linalg.norm(g_true) + 1e-9)
        p_error = np.linalg.norm(p - P_true) / (np.linalg.norm(P_true) + 1e-9)
        
        print(f'    Iter {i+1}: g_error={g_error:.4f}, p_error={p_error:.4f}')
    
    # 8. 最终结果
    g_final = g
    p_final = p
    
    # 通过Wsp得到s_output
    s_output = Wsp @ p_final
    # 归一化
    s_output = s_output / (np.linalg.norm(s_output) + 1e-9)
    
    # 计算最终误差
    g_final_error = np.linalg.norm(g_final - g_true) / (np.linalg.norm(g_true) + 1e-9)
    p_final_error = np.linalg.norm(p_final - P_true) / (np.linalg.norm(P_true) + 1e-9)
    s_final_error = np.linalg.norm(s_output - s_true) / (np.linalg.norm(s_true) + 1e-9)
    
    # 余弦相似度
    cos_sim_g = cosine_sim(g_final, g_true)
    cos_sim_p = cosine_sim(p_final, P_true)
    cos_sim_s = cosine_sim(s_output, s_true)
    
    print(f'  【最终结果】')
    print(f'    g归一化范数误差: {g_final_error:.4f}, 余弦相似度: {cos_sim_g:.4f}')
    print(f'    p归一化范数误差: {p_final_error:.4f}, 余弦相似度: {cos_sim_p:.4f}')
    print(f'    s归一化范数误差: {s_final_error:.4f}, 余弦相似度: {cos_sim_s:.4f}')
    
    # 保存结果
    results.append({
        'idx': idx,
        'obj_a': obj_a,
        'obj_b': obj_b,
        'grid_idx': grid_idx,
        'cos_sim_p_init': cos_sim_p_init,
        'g_error': g_final_error,
        'p_error': p_final_error,
        's_error': s_final_error,
        'cos_sim_g': cos_sim_g,
        'cos_sim_p': cos_sim_p,
        'cos_sim_s': cos_sim_s
    })

# 汇总结果
print('' + '='*70)
print('测试结果汇总')
print('='*70)

results_df = pd.DataFrame(results)
print(results_df[['obj_a', 'obj_b', 'g_error', 'p_error', 's_error', 'cos_sim_s']])

print(f'平均误差:')
print(f'  g归一化范数误差: {results_df["g_error"].mean():.4f}')
print(f'  p归一化范数误差: {results_df["p_error"].mean():.4f}')
print(f'  s归一化范数误差: {results_df["s_error"].mean():.4f}')

print(f'平均余弦相似度:')
print(f'  g余弦相似度: {results_df["cos_sim_g"].mean():.4f}')
print(f'  p余弦相似度: {results_df["cos_sim_p"].mean():.4f}')
print(f'  s余弦相似度: {results_df["cos_sim_s"].mean():.4f}')

print('✓ 测试完成')

测试VH权重连接：检测obj_grid_index前9个例子
测试样本数: 9
  Obj_A Obj_B  grid_index
0    稻谷    大米           1
1    核桃    桃酥          12
2    棉花    棉籽           1
3    西瓜    瓜子           1
4    花生   花生酱           1
5  旗开得胜  马到成功          16
6  拨乱反正  沉冤昭雪          16
7  牛高马大  虎穴得子          16
8  水到渠成  瓜熟蒂落          16
迭代参数:
  Niter = 5
  module_sizes = [ 9 16 25 49]
  m = 4
开始逐个测试
【样本 1】
  Obj_A: 稻谷
  Obj_B: 大米
  Grid: 1 (主体-产物)
  ✓ s_input shape: (768,), norm: 1.0000
  ✓ p_input shape: (342,), norm: 2.4222
  ✓ P_true shape: (342,), norm: 5.6644
  ✓ 初始p余弦相似度: 0.6406
  ✓ g_true shape: (99,), norm: 2.0000
  ✓ s_true (obj_b) shape: (768,), norm: 1.0000
  开始迭代优化 (Niter=5):
    Iter 1: g_error=1.0000, p_error=1.0000
    Iter 2: g_error=1.4142, p_error=1.0567
    Iter 3: g_error=1.2247, p_error=1.0000
    Iter 4: g_error=1.2247, p_error=1.0000
    Iter 5: g_error=1.4142, p_error=1.0539
  【最终结果】
    g归一化范数误差: 1.4142, 余弦相似度: 0.0000
    p归一化范数误差: 1.0539, 余弦相似度: 0.0145
    s归一化范数误差: 1.4969, 余弦相似度: -0.1204
【样本 2】
  

In [69]:
#test
print(object_to_id["巾帼"])
id = object_to_id["巾帼"]
# print(events[id])
s_true = events[id]
x, y = object_hpc_locs[id]
idx = flat_idx(x, y)
p_true = pbook_flattened[0, :, idx]
p_input = Wps @ s_true

grid_idx = 11
gbook_current = gbooks_flattened[grid_idx]
# g_in = Wgp @ p
# g = nearest_neighbor(g_in, gbook_current)
gbook_pos = gbook_obj_positions[grid_idx][obj_a_id]
g_true = gbooks_flattened[grid_idx][:, flat_idx(*gbook_pos)]
Niter =100
# print("P_true = ",p_true)
# print("P_input = ",p_input)
for i in range(Niter):
        # g_in = Wgp @ p
        g_in = Wgp_list[grid_idx] @ p_input
        
        # 使用nearest_neighbor函数，针对当前grid的gbook进行最近邻搜索
        g = nearest_neighbor(g_in, gbook_current)  # 返回 shape: (1, Ng)
        g = g.flatten()  # 转为 (Ng,)
        
        
        p = nonlin(Wpg_list[grid_idx] @ g, thresh)
print("g_cleaned = ",g)
print("g_true = ",g_true)

851
g_cleaned =  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
g_true =  [1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0.]
